In [1]:
# -*- coding: utf-8 -*-
import random
import torch


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we construct three nn.Linear instances that we will use
        in the forward pass.
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
        and reuse the middle_linear Module that many times to compute hidden layer
        representations.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same Module many
        times when defining a computational graph. This is a big improvement from Lua
        Torch, where each Module could be used only once.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            print(h_relu)
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Construct our model by instantiating the class defined above
model = DynamicNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

tensor([[0.0000, 0.0000, 0.2638,  ..., 0.4210, 0.0000, 0.2618],
        [0.0000, 0.7664, 0.0000,  ..., 0.0000, 1.3594, 0.0000],
        [0.2286, 0.0000, 0.5414,  ..., 0.0000, 0.0000, 0.2236],
        ...,
        [0.0000, 0.0000, 0.5124,  ..., 0.4168, 1.6643, 0.2113],
        [0.0000, 0.0000, 0.0000,  ..., 0.3931, 0.8214, 0.0000],
        [0.4312, 0.0000, 0.1440,  ..., 0.2892, 0.0000, 0.0000]],
       grad_fn=<ClampBackward>)
tensor([[0.2317, 0.1444, 0.2717,  ..., 0.1610, 0.3989, 0.1604],
        [0.2812, 0.2436, 0.1770,  ..., 0.1027, 0.0000, 0.3401],
        [0.0484, 0.0000, 0.3729,  ..., 0.0270, 0.0000, 0.0000],
        ...,
        [0.0000, 0.3578, 0.0000,  ..., 0.0000, 0.1853, 0.0476],
        [0.0000, 0.0619, 0.0471,  ..., 0.0560, 0.2548, 0.2585],
        [0.0737, 0.0000, 0.4842,  ..., 0.0000, 0.2373, 0.2469]],
       grad_fn=<ClampBackward>)
0 710.2177734375
tensor([[0.0000, 0.0000, 0.2613,  ..., 0.4125, 0.0000, 0.2555],
        [0.0000, 0.7650, 0.0000,  ..., 0.0000, 1.3527, 0.00